In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import get_files_counters

from os import listdir
from os.path import splitext

from IPython.display import display

# Settings

In [2]:
folder = "data\\stevci\\"
folder_ring = "data\\stevci_obvoznica\\"
use_external_weather = True

# File import

In [3]:
location_id = get_files_counters.get_ids(folder)
id_location = {location_id[x]:x for x in location_id}

In [4]:
columns = ["STM", 
           "date" ,
           "time", 
           "motorbike1", 
           "car1", 
           "bus1", 
           "LT1", 
           "ST1", 
           "TT1", 
           "TP1", 
           "TPP1", 
           "count1", 
           "motorbike2", 
           "car2", 
           "bus2", 
           "LT2", 
           "ST2", 
           "TT2", 
           "TP2", 
           "TPP2", 
           "count2", 
           "count"]

In [5]:
"""
file = "STEP_MOL_1001-156_2020_09_01__00_00-2020_09_30__23_50.S_U"
df = pd.read_csv(folder+file, 
                 encoding="cp1250", 
                 skiprows=12,
                 header=None, 
                 sep='\s{3,}', 
                 engine='python', 
                 usecols=np.arange(len(columns)))
"""

'\nfile = "STEP_MOL_1001-156_2020_09_01__00_00-2020_09_30__23_50.S_U"\ndf = pd.read_csv(folder+file, \n                 encoding="cp1250", \n                 skiprows=12,\n                 header=None, \n                 sep=\'\\s{3,}\', \n                 engine=\'python\', \n                 usecols=np.arange(len(columns)))\n'

# Parsing

## From CSV files

In [6]:
i = location_id["Dunajska zel. podvoz > Center"]
del location_id["Dunajska zel. podvoz > Center"]
location_id["Dunajska zel. podvoz <obe smeri> Center"] = i

In [7]:
for i1,name in enumerate(location_id):
    files = get_files_counters.get_file_names(name, location_id, folder)      
    
    for i2,f in enumerate(files):
        df22 = pd.read_csv(folder+f, encoding="cp1250", skiprows=12,sep='\s{1,}', header=None, engine='python', usecols=np.arange(len(columns)))
        #df = pd.read_csv(folder+f, encoding="cp1250", skiprows=11,sep='\s{1,}', engine='python')
        if len(df22.iloc[0,1])<8: # pri letališki je oznaka 1063 Let in je treba en stolpec izpustiti
            df22 = pd.read_csv(folder+f, encoding="cp1250", skiprows=12,sep='\s{1,}', header=None, engine='python', usecols=np.arange(len(columns)+1))
            df22 = df22.drop(columns=df22.columns[1])    
        
        df22 = df22.dropna()
        df22.columns = columns
        if "obe smeri" not in name: 
            df22 = df22[['date', 'time', 'count']]
        else:
            df22 = df22[['date', 'time', 'count1', 'count2']]
            
        if i2 == 0:
            df21 = df22
        else:
            df21 = pd.concat([df21, df22], ignore_index=True, sort=False)
   
    if "obe smeri" not in name:    
        #df21.to_csv(folder+"obdelano\\"+name.replace(">","").replace("<","")+".csv")
        
        df21['location'] = name
        df21['counter_id'] = location_id[name]
        df21['direction'] = 0
        if i1 == 0:
            df = df21
        else:
            df = pd.concat([df, df21], ignore_index=True, sort=False)
    else:
        name_smer1 = name.replace("<obe smeri>","SMER 0")
        name_smer2 = name.replace("<obe smeri>","SMER 1")
            
        df_smer1 = df21[['date', 'time', 'count1']].copy()
        df_smer2 = df21[['date', 'time', 'count2']].copy()
        
        df_smer1.columns = ['date', 'time', 'count']
        df_smer2.columns = ['date', 'time', 'count']
        
        #df_smer1.to_csv(folder+"obdelano\\"+name_smer1+".csv")
        #df_smer2.to_csv(folder+"obdelano\\"+name_smer2+".csv")
    
        df_smer1['location'] = name_smer1
        df_smer1['counter_id'] = location_id[name]
        df_smer1['direction'] = 0
        
        df_smer2['location'] = name_smer2
        df_smer2['counter_id'] = location_id[name]
        df_smer2['direction'] = 1
                
        if i1 == 0:
            df = pd.concat([df_smer1, df_smer2], ignore_index=True, sort=False)
        else:
            df = pd.concat([df, df_smer1, df_smer2], ignore_index=True, sort=False)
        

## From additional CSV files

In [8]:
folder_additional = "data\\stevci_dodatni\\"
l = listdir(folder_additional)

name = l[0]

file_name = folder_additional + name
    
f = open(file_name, encoding="cp1250")
f = f.readlines()
label = f[2].split(":")[1].split("/")[0].strip()
cols = f[22].strip().split()

location = label.split()[1:]
counter_id = name.split("_")[0]



In [9]:
folder_additional = "data\\stevci_dodatni\\"
l = listdir(folder_additional)

for i1,name in enumerate(l):
    file_name = folder_additional + name
    
    f = open(file_name, encoding="cp1250")
    f = f.readlines()
    label = f[2].split(":")[1].split("/")[0].strip()
    cols = f[22].strip().split()

    location = label.split()[1:]
    counter_id = name.split("_")[0]
    
    df2 = pd.read_csv(file_name, encoding="cp1250", skiprows=24,sep='\s{1,}', header=None, engine='python', usecols=np.arange(len(cols)))
    df2 =df2.iloc[:,[1,2,11,20,21]]
    df2.columns = ["date", "time", "count1", "count2", "count"]
    df2 = df2.dropna()
    
    if "obe_smeri" not in name:    
        df2 = df2[['date', 'time', 'count']]
        df2['location'] = location
        df2['counter_id'] = counter_id
        df2['direction'] = 0
        df = pd.concat([df, df2], ignore_index=True, sort=False)
    
    else:
        df2 = df2[['date', 'time', 'count1', 'count2']]
        
        name_smer1 = name + " SMER 0"
        name_smer2 = name + " SMER 1"
            
        df_smer1 = df2[['date', 'time', 'count1']].copy()
        df_smer2 = df2[['date', 'time', 'count2']].copy()
        
        df_smer1.columns = ['date', 'time', 'count']
        df_smer2.columns = ['date', 'time', 'count']
        
        #df_smer1.to_csv(folder+"obdelano\\"+name_smer1+".csv")
        #df_smer2.to_csv(folder+"obdelano\\"+name_smer2+".csv")
    
        df_smer1['location'] = name_smer1
        df_smer1['counter_id'] = counter_id
        df_smer1['direction'] = 0
        
        df_smer2['location'] = name_smer2
        df_smer2['counter_id'] = counter_id
        df_smer2['direction'] = 1
                
        df = pd.concat([df, df_smer1, df_smer2], ignore_index=True, sort=False)
        
        

In [10]:
df = df[df["date"]!="skupaj"].copy()

In [11]:
df['date'] = df['date'].map(lambda x: "20"+x.split('.')[2]) + "-" + df['date'].map(lambda x: x.split('.')[1]) + "-" + df['date'].map(lambda x: x.split('.')[0])

## From xlsx files

In [12]:
l = listdir(folder)
excel_files = []
for f in l:
    if splitext(f)[1] == ".xlsx":
        excel_files.append(f)

In [13]:
excel_files

['1002-170  Tivolska Mladika smer Vič  -  (avgust 2020).xlsx',
 '1002-170  Tivolska Mladika smer Vič  -  (oktober 2019).xlsx',
 '1005-230  Barjanska Sibirija (obe smeri)  -  (avgust 2020).xlsx',
 '1005-230  Barjanska Sibirija (obe smeri)  -  (oktober 2019).xlsx',
 '1006-220  Obvozna Roje (obe smeri)  -  (avgust 2020).xlsx',
 '1006-220  Obvozna Roje (obe smeri)  -  (oktober 2019).xlsx',
 '1008-140  Drenikova žel. Podvoz smer Šiška  -  (avgust 2020).xlsx',
 '1008-140  Drenikova žel. Podvoz smer Šiška  -  (oktober 2019).xlsx',
 '1014-150  Šmartinska 104 smer BTC  -  (avgust 2020).xlsx',
 '1014-150  Šmartinska 104 smer BTC  -  (oktober 2019).xlsx',
 '1015-120  Zaloška Toplarna smer Zalog  -  (avgust 2020).xlsx',
 '1015-120  Zaloška Toplarna smer Zalog  -  (oktober 2019).xlsx',
 '1016-140  Zaloška Toplarna smer Center  -  (avgust 2020).xlsx',
 '1016-140  Zaloška Toplarna smer Center  -  (oktober 2019).xlsx',
 '1017-246  Litijska Hrušica (obe smeri)  -  (avgust 2020).xlsx',
 '1023-266  Pot z

In [14]:
for name in excel_files:
    xls_file = pd.ExcelFile(folder+name)
    df2 = xls_file.parse(xls_file.sheet_names[1], header=None, skiprows=12)
    df2.columns = columns[1:]
    df2 = df2.dropna()
    
    counter_id = name.split()[0]
    if counter_id in id_location:
        location = id_location[counter_id]
    else:
        location = " ".join(name.split()[1:]).split("-")[0].strip()
        
    
    if "obe smeri" not in name: 
        df2 = df2[['date', 'time', 'count']]       
        df2['location'] = location
        df2['counter_id'] = counter_id
        df2['direction'] = 0
        
        df2['date'] = df2['date'].map(lambda x: x.split(".")[2]+"-"+x.split(".")[1]+"-"+x.split(".")[0])
        
        df = pd.concat([df, df2], ignore_index=True, sort=False)
    else:
        df2 = df2[['date', 'time', 'count1', 'count2']]
        
        df_smer1 = df2[['date', 'time', 'count1']].copy()
        df_smer2 = df2[['date', 'time', 'count2']].copy()
        
               
        df_smer1.columns = ['date', 'time', 'count']
        df_smer2.columns = ['date', 'time', 'count']
        
        if counter_id in id_location:
            name_smer1 = location.replace("<obe smeri>","SMER 0")
            name_smer2 = location.replace("<obe smeri>","SMER 1")
        else:
            name_smer1 = location.replace("(obe smeri)","SMER 0")
            name_smer2 = location.replace("(obe smeri)","SMER 1")
        
        
        df_smer1['location'] = name_smer1
        df_smer1['counter_id'] = counter_id
        df_smer1['direction'] = 0
        
        df_smer2['location'] = name_smer2
        df_smer2['counter_id'] = counter_id
        df_smer2['direction'] = 1
        
        df_smer1['date'] = df_smer1['date'].map(lambda x: x.split(".")[2]+"-"+x.split(".")[1]+"-"+x.split(".")[0])
        df_smer2['date'] = df_smer2['date'].map(lambda x: x.split(".")[2]+"-"+x.split(".")[1]+"-"+x.split(".")[0])
        
        df = pd.concat([df,df_smer1, df_smer2], ignore_index=True, sort=False)
  

## Additional filter - using `counter_ids.txt`

In [15]:
s=[]
f = open("data\\stevci\\counter_ids.txt", encoding="utf8")
for x in f:
    x=x.strip()
    s.extend(x.split(";"))
f.close()
print(s)
df = df[df["counter_id"].isin(s)]

['1001-156', '1014-150', '1014-176', '1016-140', '1016-14a', '1021-156', '1025-116', '1034-216', '1035-136', '1038-276', '1946-230', '1946-23a', '1947-211', '1935-230', '1935-238', '1936-211', '0174', '0855', '1010']


In [16]:
df["counter_id"].unique()

array(['1001-156', '1014-176', '1016-14a', '1021-156', '1025-116',
       '1034-216', '1038-276', '1936-211', '1946-23a', '1947-211',
       '1935-238', '0174', '0855', '1010', '1014-150', '1016-140',
       '1035-136', '1935-230', '1946-230'], dtype=object)

## Label directions

In [17]:
df["counter_id_direction"] = df["counter_id"]
df.loc[df["direction"]==1, "counter_id_direction"] = df["counter_id"]+"-1"

## Dates

In [18]:
df['date'] = pd.to_datetime(df['date'])

## Times

In [19]:
df["time"] = df["time"].map(lambda x:int(x.split(":")[0]))

# From highway (ring) counters

In [20]:
"""
df2 = pd.read_csv(folder_ring+"ring_counters.csv")
df_description = df2[["title", "id", "stevci_lokacijaOpis"]]
df_description = df_description.drop_duplicates()
df_description.to_excel(folder_ring+"ring_counters_description.xlsx", index=False)
df_description.to_csv(folder_ring+"ring_counters_description.csv", index=False)
df2 = df2[["title", "stevci_datum", "stevci_ura", "stevci_stev"]]
df2.columns = ['counter_id', 'date', 'time', 'count']
"""

'\ndf2 = pd.read_csv(folder_ring+"ring_counters.csv")\ndf_description = df2[["title", "id", "stevci_lokacijaOpis"]]\ndf_description = df_description.drop_duplicates()\ndf_description.to_excel(folder_ring+"ring_counters_description.xlsx", index=False)\ndf_description.to_csv(folder_ring+"ring_counters_description.csv", index=False)\ndf2 = df2[["title", "stevci_datum", "stevci_ura", "stevci_stev"]]\ndf2.columns = [\'counter_id\', \'date\', \'time\', \'count\']\n'

### Merge all lanes in the same direction and sum the counts within an hour

In [21]:
"""
df2['counter_id'] = df2['counter_id'].map(lambda x: x.replace(' (v)','').replace(' (p)','').replace(' (po)',''))
df2 = df2.groupby(["counter_id", "date", "time"], as_index=False).sum()

df2["counter_id_direction"] = df2["counter_id"]
df2["location"] = df2["counter_id"]
df2["direction"] = 0

df2['date'] = df2['date'].map(lambda x: x.split("/")[2]+"-"+x.split("/")[1]+"-"+x.split("/")[0])
df2['date'] = pd.to_datetime(df2['date'])

df2 = df2[['date', 'time', 'count', 'location', 'counter_id', 'direction' ,'counter_id_direction']]
"""

'\ndf2[\'counter_id\'] = df2[\'counter_id\'].map(lambda x: x.replace(\' (v)\',\'\').replace(\' (p)\',\'\').replace(\' (po)\',\'\'))\ndf2 = df2.groupby(["counter_id", "date", "time"], as_index=False).sum()\n\ndf2["counter_id_direction"] = df2["counter_id"]\ndf2["location"] = df2["counter_id"]\ndf2["direction"] = 0\n\ndf2[\'date\'] = df2[\'date\'].map(lambda x: x.split("/")[2]+"-"+x.split("/")[1]+"-"+x.split("/")[0])\ndf2[\'date\'] = pd.to_datetime(df2[\'date\'])\n\ndf2 = df2[[\'date\', \'time\', \'count\', \'location\', \'counter_id\', \'direction\' ,\'counter_id_direction\']]\n'

Rounding

In [1]:
#df2['time'] = df2['time'].map(lambda t: round(float(t.split(":")[0]) + float(t.split(":")[0])/60))

Duplicate removal

In [23]:
#df2 = df2.groupby(['date', 'time', 'location', 'counter_id', 'direction' ,'counter_id_direction'], as_index=False).max()

### Filter

In [24]:
#df2 = df2[df2["counter_id"]=="HC-H3, LJ (S obvoznica) : LJ (Celovška - Dunajska)"]

### Concat with other counters

In [25]:
df = pd.concat([df,df2], ignore_index=True, sort=False)


# Export the counter locations

In [26]:
f = open("data\\counter_locations.txt", 'w', encoding="utf8")
l = list(set(zip(df["location"], df["counter_id"])))
l.sort()
for counter_loc_id in l:
    print(counter_loc_id, file=f)
f.close()

#pd.DataFrame({'location':df.location.unique()}).to_excel("data\\counter_locations.xlsx", index=False)

# Reduce the columns

In [27]:
df = df[['date', 'time', 'counter_id_direction', 'count']]
df.columns = ['date', 'time', 'counter_id', 'count']

In [28]:
#df['date'] = df['date'].map(lambda x: "20"+x.split('.')[2]) + "-" + df['date'].map(lambda x: x.split('.')[1]) + "-" + df['date'].map(lambda x: x.split('.')[0] if len(x) < 10 else x)


# Unify the ids of the counters with different ids

In [29]:
# read the synonyms
d = {}

f = open("data\\counter_synonyms.txt", encoding="utf8")
for l in f:
    l = l.strip().split(";")
    primary = l[0]
    syns = l[1:]
    d[primary] = syns

In [30]:
for primary, syns in d.items():
    for syn in syns:
        df.loc[df['counter_id']  == syn, 'counter_id'] = primary
        df.loc[df['counter_id']  == syn+'-1', 'counter_id'] = primary+'-1'   

# Remove potential duplicates

In [33]:
df = df.groupby(['counter_id', 'date', 'time'], as_index=False).max()
# zakaj tole meč ven 1016-140;1946-230; 1035-136

# Remove outliers

In [35]:
cs = df.counter_id.unique()
cs

array(['0174', '0174-1', '0855', '0855-1', '1001-156', '1010', '1010-1',
       '1014-150', '1016-140', '1021-156', '1025-116', '1034-216',
       '1034-216-1', '1035-136', '1038-276', '1038-276-1', '1935-230',
       '1935-230-1', '1946-230', '1947-211'], dtype=object)

In [44]:
df_filtered = pd.DataFrame(columns = df.columns)


for c in cs:
    df2 = df[df.counter_id == c]
    
    
    m, s = np.mean(df2['count']), np.std(df2['count'])
    idxs = np.logical_not(np.logical_or(df2['count'] > m + 3 * s, df2['count'] < m - 3 * s))
    df2 = df2[idxs]
    df_filtered = pd.concat([df_filtered,df2], ignore_index=True, sort=False)

In [47]:
df = df_filtered

# Dates

In [48]:
df['weekday'] = df['date'].dt.weekday
df['workday'] = df['weekday'].map(lambda x: 1 if x < 5 else 0)
df = df.drop(columns = "weekday")
df['workday'].describe()

count    44552.000000
mean         0.708722
std          0.454356
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: workday, dtype: float64

# Weather data

In [49]:
df.date.head()

0   2020-10-01
1   2020-10-01
2   2020-10-01
3   2020-10-01
4   2020-10-01
Name: date, dtype: datetime64[ns]

In [50]:
#pd.to_datetime(df_weather['date']).dt.date

In [51]:
df.head()

,counter_id,date,time,count,workday
0,0174,2020-10-01,0,0.0,1
1,0174,2020-10-01,1,0.0,1
2,0174,2020-10-01,2,0.0,1
3,0174,2020-10-01,3,0.0,1
4,0174,2020-10-01,4,0.0,1


In [52]:
if not use_external_weather:
    df_weather = pd.read_csv("data\\google_data_filtered.csv", encoding="utf8")
    df_weather = df_weather[["date", "weather"]]
    df_weather = df_weather.drop_duplicates()
    df_weather["date"] = pd.to_datetime(df_weather['date'])
    df = pd.merge(df, df_weather.drop_duplicates(), on="date", how="left")
else:
    df_weather = pd.read_csv('data\\weather.csv')
    df_weather['date'] = pd.to_datetime(df_weather['date'])
    for date in df['date'].unique():
        try:
            weather = df_weather.loc[df_weather['date'] == date, 'weather'].iloc[0]
        except:
            weather = np.nan
        locs = date == df['date']
        df.loc[locs, "weather"] = weather

# Export

In [53]:
df.to_csv("data\\counters.csv", encoding="utf8", index=False)

In [54]:
df.counter_id.unique()

array(['0174', '0174-1', '0855', '0855-1', '1001-156', '1010', '1010-1',
       '1014-150', '1016-140', '1021-156', '1025-116', '1034-216',
       '1034-216-1', '1035-136', '1038-276', '1038-276-1', '1935-230',
       '1935-230-1', '1946-230', '1947-211'], dtype=object)

In [55]:
df[df['weather'].isna()].date.unique()

array([], dtype='datetime64[ns]')